In [1]:
import pandas as pd

In [11]:
df = pd.read_csv("Data.csv", delimiter = ";", skiprows = 6, skipfooter = 2, engine = "python")
df.head()

,Vervoerwijzen,Totaal,Totaal.1,Totaal.2,Totaal.3,Totaal.4,Totaal.5,Totaal.6,Totaal.7,Totaal.8,...,Overige vervoerwijze.5,Overige vervoerwijze.6,Overige vervoerwijze.7,Overige vervoerwijze.8,Overige vervoerwijze.9,Overige vervoerwijze.10,Overige vervoerwijze.11,Overige vervoerwijze.12,Overige vervoerwijze.13,Overige vervoerwijze.14
0,Onderwerp,Gemiddeld per persoon per dag |Verplaatsingen,Gemiddeld per persoon per dag |Verplaatsingen,Gemiddeld per persoon per dag |Verplaatsingen,Gemiddeld per persoon per dag |Verplaatsingen,Gemiddeld per persoon per dag |Verplaatsingen,Gemiddeld per persoon per dag |Afstand,Gemiddeld per persoon per dag |Afstand,Gemiddeld per persoon per dag |Afstand,Gemiddeld per persoon per dag |Afstand,...,Gemiddeld per persoon per dag |Afstand,Gemiddeld per persoon per dag |Afstand,Gemiddeld per persoon per dag |Afstand,Gemiddeld per persoon per dag |Afstand,Gemiddeld per persoon per dag |Afstand,Gemiddeld per persoon per dag |Reisduur,Gemiddeld per persoon per dag |Reisduur,Gemiddeld per persoon per dag |Reisduur,Gemiddeld per persoon per dag |Reisduur,Gemiddeld per persoon per dag |Reisduur
1,Perioden,2018,2019,2020,2021,2022,2018,2019,2020,2021,...,2018,2019,2020,2021,2022,2018,2019,2020,2021,2022
2,Persoonskenmerken,aantal,aantal,aantal,aantal,aantal,reizigerskilometers,reizigerskilometers,reizigerskilometers,reizigerskilometers,...,reizigerskilometers,reizigerskilometers,reizigerskilometers,reizigerskilometers,reizigerskilometers,minuten,minuten,minuten,minuten,minuten
3,Totaal personen,"2,78","2,71","2,35","2,51","2,70","36,16","36,00","24,88","27,24",...,"2,32","2,30","1,55","1,81","1,97","3,76","3,70","2,84","3,55","3,39"
4,Leeftijd:6 tot 12 jaar,"3,16","3,12","2,58","2,80","2,88","17,89","17,08","13,50","14,99",...,"0,66","0,79","0,59","0,74","0,54","2,39","2,28","3,41","5,05","2,88"


In [17]:
print(df.columns)

Index(['Vervoerwijzen', 'Totaal', 'Totaal.1', 'Totaal.2', 'Totaal.3',
       'Totaal.4', 'Totaal.5', 'Totaal.6', 'Totaal.7', 'Totaal.8',
       ...
       'Overige vervoerwijze.5', 'Overige vervoerwijze.6',
       'Overige vervoerwijze.7', 'Overige vervoerwijze.8',
       'Overige vervoerwijze.9', 'Overige vervoerwijze.10',
       'Overige vervoerwijze.11', 'Overige vervoerwijze.12',
       'Overige vervoerwijze.13', 'Overige vervoerwijze.14'],
      dtype='object', length=121)
